In [13]:
import pypsa
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import json

with open('topology_only_power.json') as json_file:
    data = json.load(json_file)

In [14]:
with open('boundary_only_power.json') as json_file:
    boundary = json.load(json_file)

In [15]:
def create_admittance(data):
    n = pypsa.Network(data)
    n.determine_network_topology()
    for sub in n.sub_networks.obj:
        pypsa.pf.calculate_Y(sub)
    
    return sub.Y

def find_controls(data):
    n = pypsa.Network(data)
    n.determine_network_topology()
    for sub in n.sub_networks.obj:
        pypsa.pf.find_bus_controls(sub)
    
    return n

In [16]:
##Erstelle Admittanzmatrix (noch mit Dateinamen hart gecodet)
x = create_admittance("elec_s_4.nc")
y = find_controls("elec_s_4.nc")

INFO:pypsa.io:Imported network elec_s_4.nc has buses, carriers, generators, lines, links, loads
INFO:pypsa.io:Imported network elec_s_4.nc has buses, carriers, generators, lines, links, loads


In [17]:
n = pypsa.Network("elec_s_4.nc")

INFO:pypsa.io:Imported network elec_s_4.nc has buses, carriers, generators, lines, links, loads


In [18]:
for sub in y.sub_networks.obj:
        print(sub.slack_bus)

DE0 0


In [30]:
admittance_matrix = pd.DataFrame.sparse.from_spmatrix(x)

In [20]:
admittance_matrix.columns =['NO', 'NW', 'SO', 'SW']
admittance_matrix.index =['NO', 'NW', 'SO', 'SW']

In [31]:
admittance_matrix

,0,1,2,3
0,3636.106022-26530.673647j,-2161.234417+17722.122217j,-946.842764+7764.110666j,-528.028842+4329.836501j
1,-2161.234417+17722.122217j,5178.484806-39413.731802j,0.000000+0.000000j,-3017.250390+24741.453195j
2,-946.842764+7764.110666j,0.000000+0.000000j,4114.683469-31804.273845j,-3167.840705+25976.293782j
3,-528.028842+4329.836501j,-3017.250390+24741.453195j,-3167.840705+25976.293782j,6713.119936-52008.976360j


In [22]:
admittance_matrix.to_dict()

{'NO': {'NO': (3636.106022465493-26530.673646506344j),
  'NW': (-2161.2344167017477+17722.122216954333j),
  'SO': (-946.8427641662366+7764.110666163139j),
  'SW': (-528.028841597509+4329.836501099575j)},
 'NW': {'NO': (-2161.2344167017477+17722.122216954333j),
  'NW': (5178.484806283053-39413.73180198083j),
  'SO': 0,
  'SW': (-3017.2503895813056+24741.45319456671j)},
 'SO': {'NO': (-946.8427641662366+7764.110666163139j),
  'NW': 0,
  'SO': (4114.683469248478-31804.2738449387j),
  'SW': (-3167.840705082241+25976.293781674383j)},
 'SW': {'NO': (-528.028841597509+4329.836501099575j),
  'NW': (-3017.2503895813056+24741.45319456671j),
  'SO': (-3167.840705082241+25976.293781674383j),
  'SW': (6713.119936261055-52008.9763595096j)}}

In [23]:
data

{'information': {'title': 'greek-power',
  'type': 'gas-power',
  'author': 'automatic conversion',
  'date': '2016-10-19',
  'documentation': 'Adapted gaslib-134 combined with ieee-300 network.'},
 'nodes': {'Vphinode': [{'B': '-26.2023789002',
    'G': '1.7238407171',
    'id': 'N213'},
   {'B': '-763.4449760692', 'G': '169.6871020263', 'id': 'N221'},
   {'B': '-30.0931800877', 'G': '0.9064210870', 'id': 'N230'},
   {'B': '-51.2032770097', 'G': '0.0000000000', 'id': 'N7001'},
   {'B': '-68.1338122127', 'G': '0.0000000000', 'id': 'N7017'},
   {'B': '-34.6020761246', 'G': '0.0000000000', 'id': 'N7024'},
   {'B': '-33.9934894479', 'G': '0.0000000000', 'id': 'N7039'},
   {'B': '-20.7225218066', 'G': '0.0000000000', 'id': 'N7057'},
   {'B': '-42.0168067227', 'G': '0.0000000000', 'id': 'N7061'},
   {'B': '-15.8501289008', 'G': '0.0000000000', 'id': 'N7071'}],
  'PVnode': [{'B': '-72.9789803995', 'G': '18.0731863544', 'id': 'N8'},
   {'B': '-29.1373612756', 'G': '0.0000000000', 'id': 'N10'}

In [24]:
# admittance to JSON

# first calculate imaginary and real parts of nodes
nodes_real = []
nodes_imag = []
nodes_names = []
for i in range(0,len(admittance_matrix)):
    nodes_real = nodes_real + [admittance_matrix.iloc[i,i].real]
    nodes_imag = nodes_imag + [admittance_matrix.iloc[i,i].imag]
    nodes_names = nodes_names + [admittance_matrix.columns[i]]
    
nodes = [0]*len(admittance_matrix)
for i in range(0,len(admittance_matrix)):
    nodes[i] = {'B':nodes_imag[i].astype(str), 'G':nodes_real[i].astype(str), 'id':nodes_names[i]}

# calculate real and imaginary parts of transmissionlines
transmission_line_real = []
transmission_line_imag = []
transmission_line_from = []
transmission_line_to = []
transmission_line_id = []
for i in range(0, len(admittance_matrix)):
    for j in range(0, len(admittance_matrix)):
        if not i == j:
            transmission_line_real = transmission_line_real + [np.float64(admittance_matrix.iloc[i,j].real)]
            transmission_line_imag = transmission_line_imag + [np.float64(admittance_matrix.iloc[i,j].imag)]
            transmission_line_from = transmission_line_from + [admittance_matrix.index[i]]
            transmission_line_to = transmission_line_to + [admittance_matrix.columns[j]]
            transmission_line_id = transmission_line_id + [admittance_matrix.index[i]+"_"+admittance_matrix.columns[j]]

transmission_lines = [0]*(pow(len(admittance_matrix),2)-len(admittance_matrix))
for i in range(0,pow(len(admittance_matrix),2)-len(admittance_matrix)):
    transmission_lines[i] = {'B':transmission_line_imag[i].astype(str), 
                             'G':transmission_line_real[i].astype(str), 
                             'from':transmission_line_from[i],
                             'id':transmission_line_id[i],
                             'to':transmission_line_to[i]}
    
# store results in JSON Format    
admittance_in_json = {"nodes": {"PQnode":nodes}, "connections": {"Transmissionline":transmission_lines}}
#admittance_in_json = str(admittance_in_json)
#admittance_in_json = json.loads(admittance_in_json)
admittance_in_json = json.dumps(admittance_in_json, indent=4, sort_keys=True)

In [25]:
print(json.loads(admittance_in_json))

{'connections': {'Transmissionline': [{'B': '17722.122216954333', 'G': '-2161.2344167017477', 'from': 'NO', 'id': 'NO_NW', 'to': 'NW'}, {'B': '7764.110666163139', 'G': '-946.8427641662366', 'from': 'NO', 'id': 'NO_SO', 'to': 'SO'}, {'B': '4329.836501099575', 'G': '-528.028841597509', 'from': 'NO', 'id': 'NO_SW', 'to': 'SW'}, {'B': '17722.122216954333', 'G': '-2161.2344167017477', 'from': 'NW', 'id': 'NW_NO', 'to': 'NO'}, {'B': '0.0', 'G': '0.0', 'from': 'NW', 'id': 'NW_SO', 'to': 'SO'}, {'B': '24741.45319456671', 'G': '-3017.2503895813056', 'from': 'NW', 'id': 'NW_SW', 'to': 'SW'}, {'B': '7764.110666163139', 'G': '-946.8427641662366', 'from': 'SO', 'id': 'SO_NO', 'to': 'NO'}, {'B': '0.0', 'G': '0.0', 'from': 'SO', 'id': 'SO_NW', 'to': 'NW'}, {'B': '25976.293781674383', 'G': '-3167.840705082241', 'from': 'SO', 'id': 'SO_SW', 'to': 'SW'}, {'B': '4329.836501099575', 'G': '-528.028841597509', 'from': 'SW', 'id': 'SW_NO', 'to': 'NO'}, {'B': '24741.45319456671', 'G': '-3017.2503895813056', '

In [123]:
with open('test36.txt', 'w') as json_file:
  json.dump(admittance_in_json, json_file)

In [39]:
with open('json_admittance.json', 'w') as json_file:
    json.dump(admittance_in_json, json_file)

In [38]:
admittance_in_json

'{\n    "connections": {\n        "Transmissionline": [\n            {\n                "B": "17722.122216954333",\n                "G": "-2161.2344167017477",\n                "from": "NO",\n                "id": "NO_NW",\n                "to": "NW"\n            },\n            {\n                "B": "7764.110666163139",\n                "G": "-946.8427641662366",\n                "from": "NO",\n                "id": "NO_SO",\n                "to": "SO"\n            },\n            {\n                "B": "4329.836501099575",\n                "G": "-528.028841597509",\n                "from": "NO",\n                "id": "NO_SW",\n                "to": "SW"\n            },\n            {\n                "B": "17722.122216954333",\n                "G": "-2161.2344167017477",\n                "from": "NW",\n                "id": "NW_NO",\n                "to": "NO"\n            },\n            {\n                "B": "0.0",\n                "G": "0.0",\n                "from": "NW",\n

In [7]:
boundary

{'nodes': {'Vphinode': [{'id': 'N213',
    'data': [{'time': 0, 'values': [1.01, -0.2036799237]},
     {'time': 86400, 'values': [1.01, -0.2036799237]}]},
   {'id': 'N221',
    'data': [{'time': 0, 'values': [1.0, -0.3925245488]},
     {'time': 86400, 'values': [1.0, -0.3925245488]}]},
   {'id': 'N230',
    'data': [{'time': 0, 'values': [1.04, -0.2412045026]},
     {'time': 86400, 'values': [1.04, -0.2412045026]}]},
   {'id': 'N7001',
    'data': [{'time': 0, 'values': [1.0507, 0.1883210263]},
     {'time': 86400, 'values': [1.0507, 0.1883210263]}]},
   {'id': 'N7017',
    'data': [{'time': 0, 'values': [1.0507, -0.1827359727]},
     {'time': 86400, 'values': [1.0507, -0.1827359727]}]},
   {'id': 'N7024',
    'data': [{'time': 0, 'values': [1.029, 0.2199114858]},
     {'time': 86400, 'values': [1.029, 0.2199114858]}]},
   {'id': 'N7039',
    'data': [{'time': 0, 'values': [1.05, 0.0368264472]},
     {'time': 86400, 'values': [1.05, 0.0368264472]}]},
   {'id': 'N7057',
    'data': [{'t